In [2]:
import requests
import statsmodels.formula.api as smf    # for OLS regression
import pylab as pl
import pandas as pd
import numpy as np
import json
import urllib 
import scipy.stats
import os
import sys
%pylab inline

#importing geopandas read to plot geographical information
import geopandas as gpd
#importing fiona to handle geographical coordinates
import fiona
#import shapely to handle geographical shapes
import shapely
import urllib.request
import gzip
from shapely.geometry import Point
from geopandas.tools import sjoin


Populating the interactive namespace from numpy and matplotlib


# Yellow Cab

# files

In [5]:
for year in range(17,19):
    for i in range(1,13):
        os.system('curl -O https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_' + str(2000+year) + '-' + str(i).zfill(2) + '.csv')
        os.system('mv yellow_tripdata_'+ str(2000+year) + '-' + str(i).zfill(2) + '.csv ' + '/Volumes/MACExt/FloodHousing/yellowcab')

In [ ]:
os.system("curl -O https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2009-02.csv")
os.system("mv yellow_tripdata_2009-02.csv " + os.getenv("PUIDATA"))

In [ ]:
import glob
#path = r'Data/FHV2015'                     # use your path
all_files = glob.glob(os.path.join("*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
#concatenated_df   =  pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
len(all_files)

# Census

In [2]:
census = gpd.GeoDataFrame.from_file('cb_2016_36_tract_500k/cb_2016_36_tract_500k.shp')
census.crs

{'init': 'epsg:4269'}

# Get Floodzone FIRM07 and PFIRM15 Census Tracts 

In [ ]:
floodzone07 = pd.read_csv('/Users/cihe/Documents/CUSPClassAssignments/Capstone/Code/pluto_FIRM07.csv')

In [ ]:
floodzone07.drop([u'Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
floodzone07.head(10)

In [ ]:
floodzone07.CT2010 = floodzone07.CT2010*100

In [ ]:
floodzone07.CT2010 = floodzone07.CT2010.fillna(0)

In [ ]:
floodzone07.CT2010 = floodzone07.CT2010.astype('int')

In [ ]:
floodzone07.CT2010 = floodzone07.CT2010.astype('str')
floodzone07.CT2010 = floodzone07.CT2010.str.zfill(6)

In [ ]:
floodzone07.CT2010 = floodzone07.CT2010.astype('int')
floodzone07.CT2010 = floodzone07.CT2010 + floodzone07.Boro*1000000

In [ ]:
floodzone07.head()

In [ ]:
floodzone07.rename(columns={'CT2010':'GEOID'}, inplace=True)


In [ ]:
floodzone07.GEOID = floodzone07.GEOID.astype('str')


In [ ]:
#floodzone07.to_csv('floodzone07_wGEOID.csv')

In [ ]:
tract_floodzone07 = gpd.GeoDataFrame.merge(census, floodzone07, how='inner', on='GEOID')

In [ ]:
tract_floodzone07.plot()

In [ ]:
floodzone15 = pd.read_csv('/Users/cihe/Documents/CUSPClassAssignments/Capstone/Code/pluto_PFIRM15.csv')

In [ ]:
floodzone15.drop([u'Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
floodzone15.CT2010 = floodzone15.CT2010*100
floodzone15.CT2010 = floodzone15.CT2010.fillna(0)
floodzone15.CT2010 = floodzone15.CT2010.astype('int')
floodzone15.CT2010 = floodzone15.CT2010.astype('str')
floodzone15.CT2010 = floodzone15.CT2010.str.zfill(6)
floodzone15.CT2010 = floodzone15.CT2010.astype('int')
floodzone15.CT2010 = floodzone15.CT2010 + floodzone15.Boro*1000000

In [ ]:
floodzone15.head()

In [ ]:
floodzone15.rename(columns={'CT2010':'GEOID'}, inplace=True)
floodzone15.GEOID = floodzone15.GEOID.astype('str')

In [ ]:
#floodzone15.to_csv('floodzone15_wGEOID.csv')

In [ ]:
tract_floodzone15 = gpd.GeoDataFrame.merge(census, floodzone15, how='inner', on='GEOID')


In [ ]:
tract_floodzone15.plot()

# Convert Lat Lon Info to  GeoPoint

In [ ]:
all_files

In [ ]:
#df = pd.read_csv(all_files[12])


In [ ]:
#df.columns

In [ ]:
#(' pickup_datetime'  in df.columns) & (' pickup_longitude' in df.columns) & (' pickup_latitude' in df.columns)

In [ ]:
#for i in range(len(all_files)):
for i in range(19,24):
    df = pd.read_csv(all_files[i])
    #downsample
    df = df.sample(frac=0.02, replace=True)
    if ((' pickup_datetime'  in df.columns) & (' pickup_longitude' in df.columns) & (' pickup_latitude' in df.columns)):
        df = df[[' pickup_datetime', ' pickup_longitude', ' pickup_latitude']]
        df[' pickup_latitude'].astype('float')
        df[' pickup_longitude'].astype('float')
        df['geometry'] = df.apply(lambda z: Point(z[' pickup_longitude'], z[' pickup_latitude']), axis=1)
    else:
        df = df[['pickup_datetime', 'pickup_longitude', 'pickup_latitude']]
        df.pickup_latitude.astype('float')
        df.pickup_longitude.astype('float')
        df['geometry'] = df.apply(lambda z: Point(z['pickup_longitude'], z['pickup_latitude']), axis=1)
    pickup_point = df.geometry
    pickup_point = gpd.GeoDataFrame(pickup_point)
    pickup_point.crs = tract_floodzone07.crs
    pointInPolys_07_pickup = sjoin(pickup_point, tract_floodzone07, how='left')
    FIRM07_pickup = pointInPolys_07_pickup.groupby('GEOID').count()
    FIRM07_pickup.reset_index(inplace=True)
    FIRM07_pickup = FIRM07_pickup[['GEOID','index_right']]
    FIRM07_pickup.rename(columns={'index_right':'TAXI_PICKUP'}, inplace=True)
    floodzone07_taxi = floodzone07.merge(FIRM07_pickup, how='inner', on='GEOID')
    floodzone07_taxi.to_csv('floodzone07_taxi' + str(i) + '.csv')
    
    pointInPolys_15_pickup = sjoin(pickup_point, tract_floodzone15, how='left')
    PFIRM15_pickup = pointInPolys_15_pickup.groupby('GEOID').count()
    PFIRM15_pickup.reset_index(inplace=True)
    PFIRM15_pickup = PFIRM15_pickup[['GEOID','index_right']]
    PFIRM15_pickup.rename(columns={'index_right':'TAXI_PICKUP'}, inplace=True)
    floodzone15_taxi = floodzone15.merge(PFIRM15_pickup, how='inner', on='GEOID')
    floodzone15_taxi.to_csv('floodzone15_taxi' + str(i) + '.csv')

In [ ]:
df = pd.read_csv(all_files[0])
df = df[['pickup_datetime', 'dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 
         'dropoff_latitude']]
df.pickup_latitude.astype('float')
df.pickup_longitude.astype('float')
df.dropoff_longitude.astype('float')
df.dropoff_latitude.astype('float')

In [ ]:
df['geometry1'] = df.apply(lambda z: Point(z['pickup_longitude'], z['pickup_latitude']), axis=1)
pickup_point = df.geometry1
pickup_point = gpd.GeoDataFrame(pickup_point)
df['geometry2'] = df.apply(lambda z: Point(z['dropoff_longitude'], z['dropoff_latitude']), axis=1)
dropoff_point = df.geometry2
dropoff_point = gpd.GeoDataFrame(dropoff_point)

In [ ]:
pickup_point.crs = tract_floodzone07.crs

In [ ]:
pickup_point.rename(columns={'geometry1':'geometry'}, inplace=True)

In [ ]:
tract_floodzone07.drop([u'Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
pointInPolys_07_pickup = sjoin(pickup_point, tract_floodzone07, how='left')
FIRM07_pickup = pointInPolys_07_pickup.groupby('GEOID').sum()
FIRM07_pickup.reset_index(inplace=True)

In [ ]:
FIRM07_pickup.head()

In [ ]:
pickup_point.shape

In [ ]:
len(concatenated_df)

In [ ]:
concatenated_df.shape

In [ ]:
concatenated_df.dropna(axis=0, inplace=True)
concatenated_df.shape

In [ ]:
concatenated_df.to_csv('Data/FHV2015/FHV2015_concat.csv')

In [ ]:
concatenated_df.head()

#init from concat file

In [ ]:
FHV = pd.read_csv('Data/FHV2015/FHV2015_concat.csv')

In [ ]:
FHV.shape

In [ ]:
FHV.head()

In [ ]:
FHV.reset_index(inplace=True)
FHV.rename(columns={'index':'dispatching_number'}, inplace=True)


In [ ]:
FHV.drop([u'Unnamed: 0', u'Dispatching_base_num'], axis = 1, inplace = True)

In [ ]:
FHV.to_csv('Data/FHV/FHV2016.csv')

# project taxi zone to tract

In [ ]:
#census = gpd.GeoDataFrame.from_file('Data/cb_2016_36_tract_500k/cb_2016_36_tract_500k.shp')
#census.head()

NYCcensus = gpd.GeoDataFrame.from_file('Data/nyc tracts/nyu_2451_34513.shp')
NYCcensus.head()

In [ ]:
NYCcensus.crs

In [ ]:
NYCcensus.plot(facecolor='w',edgecolor='k')

In [ ]:
taxizone = gpd.GeoDataFrame.from_file('Data/taxi zone/taxi_zones.shp')
taxizone.head()

In [ ]:
taxizone.crs

In [ ]:
taxizone.plot(figsize(10,10),facecolor='w',edgecolor='k')

In [ ]:
NYCcensus.crs = taxizone.crs

In [ ]:
zonecombined = gpd.sjoin(NYCcensus,taxizone,how="left",op="intersects")
zonecombined.fillna(0, inplace = True)
zonecombined

In [ ]:
zonecombined = zonecombined[zonecombined.index_right>0]
zonecombined

In [ ]:
def get_size_of_intersection(base, taxi):
    return base['geometry'].intersection(taxi['geometry'].iloc[int(base['index_right'])]).area



In [ ]:
zonecombined['intersection_size'] = zonecombined.apply(lambda row : 
                                       get_size_of_intersection(row, taxizone), axis=1)

In [ ]:
zonecombined

In [ ]:
zonecombined.columns

In [ ]:
zonecombined.drop([u'statefp', u'countyfp', u'name', u'namelsad', u'mtfcc',
       u'funcstat', u'aland', u'awater', u'OBJECTID'], axis = 1, inplace = True)

In [ ]:
zonecombined.head()

In [ ]:
taxizone['area'] = taxizone.geometry.area
taxizone

In [ ]:
zonecombined.reset_index(inplace=True)

In [ ]:
zonecombined

In [ ]:
zonecombined['taxizone_area']=0
for k in range(len(zonecombined)):
        zonecombined['taxizone_area'][k] = taxizone['area'][int(zonecombined['LocationID'][k]-1)]



In [ ]:
zonecombined.rename(columns={"index": "zone_percentage"},inplace=True)

In [ ]:
zonecombined['zone_percentage']=zonecombined['intersection_size']/zonecombined['taxizone_area']

In [ ]:
zonecombined

In [ ]:
zonecombined.to_csv('Data/combinezonefinal.csv', sep=',', index=True)

# get each tracts trip quantities

#format cleaning

In [ ]:
FHV = pd.read_csv('Data/FHV/FHV2016.csv')

In [ ]:
FHV.head()

In [ ]:
FHV['Pickup_date'] = pd.to_datetime(FHV['Pickup_date'])

In [ ]:
FHV.Pickup_date[6]

In [ ]:
FHV.drop([u'Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
FHV.to_csv('Data/FHV/FHV2016cleaned.csv')

#use cleaned data

In [ ]:
FHV = pd.read_csv('Data/FHV/FHV2016cleaned.csv')
FHV.head()

In [ ]:
FHV_quan = FHV.groupby(['locationID']).count()

In [ ]:
FHV_quan.shape

In [ ]:
FHV_quan.reset_index(inplace=True)


In [ ]:
FHV_quan.head()

In [ ]:
FHV_quan.rename(columns={'Pickup_date':'pickup'}, inplace=True)



In [ ]:
FHV_quan.drop([u'Pickup_date'], axis = 1, inplace = True)


In [ ]:
FHV_quan.drop([u'index'], axis = 1, inplace = True)
FHV_quan.head()

In [ ]:
FHV_quan.shape

In [ ]:
FHV_quan.rename(columns={'locationID':'LocationID'}, inplace=True)
FHV_quan.to_csv('Data/FHV2015/FHV_quan.csv')

# find FHV picks for each tracts

In [ ]:
intracts = pd.read_csv('Data/combinezonefinal.csv')
intracts.head()

In [ ]:
intracts.shape

In [ ]:
FHV_quan = pd.read_csv('Data/FHV/FHV_quan.csv')

In [ ]:
FHV_quan.head()

In [ ]:
type(FHV_quan.LocationID[6])

In [ ]:
type(intracts.LocationID[6])

In [ ]:
intractsmerge = pd.merge(intracts, FHV_quan, how='inner', on='LocationID')

In [ ]:
intractsmerge.head()

In [ ]:
intractsmerge.rename(columns={'Unnamed: 0':'pickups_intract'}, inplace=True)

In [ ]:
intractsmerge['pickups_intract'] = intractsmerge.zone_percentage*intractsmerge.pickup

In [ ]:
heat = intractsmerge.groupby(['geoid']).sum()

In [ ]:
heat.head()

In [ ]:
heat.columns

In [ ]:
heat.reset_index(inplace=True)
heat.head()


In [ ]:
heat.drop([ u'zone_percentage', u'tractce',u'intptlat', u'intptlon',
       u'index_right', u'Shape_Leng', u'Shape_Area', u'LocationID',
       u'intersection_size', u'taxizone_area', u'pickup'], axis = 1, inplace = True)

In [ ]:
#heat.drop([u'index',], axis = 1, inplace = True)
heat.head()

In [ ]:
heat.shape

In [ ]:
heat.to_csv('Data/FHV2015/pickups2015.csv')

In [ ]:
#plot heatmap on census

In [ ]:
NYCcensus = gpd.GeoDataFrame.from_file('Data/nyc tracts/nyu_2451_34513.shp')
NYCcensus.head()

In [ ]:
NYCcensus['geoid'] = NYCcensus['geoid'].astype(int)

In [ ]:
NYCcensusmerge = gpd.GeoDataFrame.merge(NYCcensus, heat, how='inner', on='geoid')

In [ ]:
NYCcensusmerge

In [ ]:
NYCcensusmerge.plot(column='pickups_intract',cmap='OrRd', scheme='quantiles',facecolor='w',edgecolor='k')

In [ ]:
NYCcensusmergeFHV = NYCcensusmerge[['geoid','pickups_intract']]

In [ ]:
NYCcensusmergeFHV.to_csv('Data/FHV/NYCcensusmergeFHV.csv')

In [ ]:
import choroplethNYC as cp

# find latlon from address

In [ ]:
import urllib
import json

googleGeocodeUrl = 'http://maps.googleapis.com/maps/api/geocode/json?'

def get_coordinates(query, from_sensor=False):
    query = query.encode('utf-8')
    params = "address={}".format(
            query,
#        'sensor': "true" if from_sensor else "false"
        )
    url = "{googleGeocodeUrl}{params}".format(googleGeocodeUrl=googleGeocodeUrl, params=params)
    response = requests.get(url)
    if response.json()['results']:
        location = response.json()['results'][0]['geometry']['location']
        latitude, longitude = location['lat'], location['lng']
#        print (query, latitude, longitude)
    else:
        latitude, longitude = None, None
        print (query, "<no results>")
    return latitude, longitude

In [ ]:
a = '2031 spring cedar ln, houston'
get_coordinates(a)[0]


In [ ]:
newbuildings = pd.read_excel("Data/brooklynnewbuildings.xlsx")

In [ ]:
len(newbuildings)

In [ ]:
for i in newbuildings.index:
    newbuildings.loc[i,'lat'] = get_coordinates(newbuildings.loc[i,'full_address'])[0]
    newbuildings.loc[i,'lon'] = get_coordinates(newbuildings.loc[i,'full_address'])[1]

In [ ]:
newbuildings.to_csv('newbuildingsloc.csv')